In [44]:
import pandas as pd

In [45]:
df=pd.read_csv("train.csv")

In [46]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [47]:
df=df.dropna()

In [48]:
x=df.drop('label',axis=1)

In [49]:
y=df['label']

In [50]:
x.shape

(18285, 4)

In [51]:
y.shape

(18285,)

In [52]:
import tensorflow as tf

In [53]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [54]:
voc_size=5000

In [55]:
msg=x.copy()

In [56]:
msg.reset_index(inplace=True)

In [57]:
import nltk
import re
from nltk.corpus import stopwords


In [59]:
from nltk.stem import PorterStemmer


In [69]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(msg)):
  
  review=re.sub('[^a-zA-Z]',' ',msg['title'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [70]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [72]:
#onehot_repr

In [73]:
sent_len=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_len)
print(embedded_docs)

[[   0    0    0 ... 2397 2997 3764]
 [   0    0    0 ... 3237  210 3041]
 [   0    0    0 ... 1261  199 1351]
 ...
 [   0    0    0 ... 3046 1952 1581]
 [   0    0    0 ... 4159 1656 4954]
 [   0    0    0 ... 3189 2451 1702]]


In [74]:
len(embedded_docs)

18285

In [80]:
#creating the model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [83]:
import numpy as np
x_final=np.array(embedded_docs)
y_final=np.array(y)

In [82]:
x_final.shape

(18285, 20)

In [85]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.2,random_state=0)

In [86]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
229/229 [==============================] - 10s 33ms/step - loss: 0.4491 - accuracy: 0.7686 - val_loss: 0.1999 - val_accuracy: 0.9204
Epoch 2/10
229/229 [==============================] - 7s 31ms/step - loss: 0.1365 - accuracy: 0.9462 - val_loss: 0.1843 - val_accuracy: 0.9237
Epoch 3/10
229/229 [==============================] - 7s 30ms/step - loss: 0.0940 - accuracy: 0.9699 - val_loss: 0.2232 - val_accuracy: 0.9251
Epoch 4/10
229/229 [==============================] - 7s 30ms/step - loss: 0.0721 - accuracy: 0.9760 - val_loss: 0.2134 - val_accuracy: 0.9226
Epoch 5/10
229/229 [==============================] - 7s 30ms/step - loss: 0.0556 - accuracy: 0.9819 - val_loss: 0.2259 - val_accuracy: 0.9161
Epoch 6/10
229/229 [==============================] - 7s 30ms/step - loss: 0.0436 - accuracy: 0.9888 - val_loss: 0.2638 - val_accuracy: 0.9240
Epoch 7/10
229/229 [==============================] - 7s 31ms/step - loss: 0.0268 - accuracy: 0.9924 - val_loss: 0.3129 - val_accuracy: 0.915

In [87]:
y_predict=model.predict_classes(x_test)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [88]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_predict)

In [89]:
cm

array([[1862,  178],
       [ 117, 1500]])

In [90]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.9193327864369701